In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from matplotlib import pyplot as plt
import seaborn as sns
import joblib

In [2]:
!ls


data	      encoder5.pkl  X_scaler1.pkl  X_scaler6.pkl  Y_scaler5.pkl
encoder1.pkl  encoder6.pkl  X_scaler2.pkl  Y_scaler1.pkl  Y_scaler6.pkl
encoder2.pkl  encoder.pkl   X_scaler3.pkl  Y_scaler2.pkl
encoder3.pkl  Note1.ipynb   X_scaler4.pkl  Y_scaler3.pkl
encoder4.pkl  trials.ipynb  X_scaler5.pkl  Y_scaler4.pkl


In [3]:
df= pd.read_csv('data/Global_EV_2023.csv')

In [4]:
df.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00046
3,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.00000


In [5]:
cat_cols=[]
num_cols=[]
for cols in df.columns:
    if df[cols].dtype=="object":
        cat_cols.append(cols)
    else:
        num_cols.append(cols)

In [6]:
num_cols

['year', 'value']

In [7]:
cat_cols

['region', 'category', 'parameter', 'mode', 'powertrain', 'unit']

In [8]:
for cat in cat_cols:
    print(df[cat].isnull().sum())

0
0
0
0
0
0


In [9]:
for cat in num_cols:
    print(df[cat].isnull().sum())

0
0


In [10]:
df[cat_cols].nunique()

region        38
category       3
parameter      8
mode           5
powertrain     5
unit           6
dtype: int64

In [11]:
for cols in cat_cols:
    print(cols)
    print(df[cols].unique())
    print()

region
['Australia' 'Austria' 'Belgium' 'Brazil' 'Canada' 'Chile' 'China'
 'Denmark' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece' 'Iceland'
 'India' 'Indonesia' 'Israel' 'Italy' 'Japan' 'Korea' 'Mexico'
 'Netherlands' 'New Zealand' 'Norway' 'Other Europe' 'Poland' 'Portugal'
 'Rest of the world' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Thailand' 'Turkiye' 'United Kingdom' 'USA' 'World']

category
['Historical' 'Projection-STEPS' 'Projection-APS']

parameter
['EV stock' 'EV sales share' 'EV stock share' 'EV sales'
 'EV charging points' 'Electricity demand' 'Oil displacement Mbd'
 'Oil displacement, million lge']

mode
['Cars' 'EV' 'Buses' 'Vans' 'Trucks']

powertrain
['BEV' 'EV' 'PHEV' 'Publicly available fast' 'Publicly available slow']

unit
['Vehicles' 'percent' 'charging points' 'GWh' 'Milion barrels per day'
 'Oil displacement, million lge']



In [12]:
# from sklearn.preprocessing import OrdinalEncoder
# encoder = OrdinalEncoder()
# df[cat_cols]=encoder.fit_transform(df[cat_cols])

In [13]:
for cat in num_cols:
    IQR=df[cat].quantile(0.75)-df[cat].quantile(0.25)
    UB= df[cat].quantile(0.75)+ (1.5*IQR)
    LB= df[cat].quantile(0.25)-(1.5*IQR)
    s=0
    for row in df[cat]:
        if row>=UB or row<= LB:
            s+=1
    print(f"{cat}: {s}")

year: 444
value: 1826


In [14]:
df.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00046
3,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.00000


In [15]:
df1 = df[df["unit"]=="Vehicles"]

In [16]:
df2 = df[df["unit"]=="percent"]

In [17]:
df3 = df[df["unit"]=="charging points"]

In [18]:
df4 = df[df["unit"]=="GWh"]

In [19]:
df5 = df[df["unit"]=="Milion barrels per day"]

In [20]:
df6 = df[df["unit"]=="Oil displacement, million lge"]

In [21]:
for it in cat_cols:
    
    # if df1[it].dtype == 'object':
        print(df1[it].unique())

['Australia' 'Austria' 'Belgium' 'Brazil' 'Canada' 'Chile' 'China'
 'Denmark' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece' 'Iceland'
 'India' 'Israel' 'Italy' 'Japan' 'Korea' 'Mexico' 'Netherlands'
 'New Zealand' 'Norway' 'Other Europe' 'Poland' 'Portugal'
 'Rest of the world' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Turkiye' 'United Kingdom' 'USA' 'World']
['Historical' 'Projection-STEPS' 'Projection-APS']
['EV stock' 'EV sales']
['Cars' 'Buses' 'Vans' 'Trucks']
['BEV' 'PHEV']
['Vehicles']


In [22]:
df2.head()

,region,category,parameter,mode,powertrain,year,unit,value
1,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
2,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00046
5,Australia,Historical,EV stock share,Cars,EV,2012,percent,0.00280
6,Australia,Historical,EV sales share,Cars,EV,2012,percent,0.03000
13,Australia,Historical,EV sales share,Cars,EV,2013,percent,0.03400


In [23]:
for i in df2.columns:
    print(df2[i].unique())

['Australia' 'Austria' 'Belgium' 'Brazil' 'Canada' 'Chile' 'China'
 'Denmark' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece' 'Iceland'
 'India' 'Israel' 'Italy' 'Japan' 'Korea' 'Mexico' 'Netherlands'
 'New Zealand' 'Norway' 'Other Europe' 'Poland' 'Portugal'
 'Rest of the world' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Turkiye' 'United Kingdom' 'USA' 'World']
['Historical' 'Projection-APS' 'Projection-STEPS']
['EV sales share' 'EV stock share']
['Cars' 'Buses' 'Vans' 'Trucks']
['EV']
[2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2010 2025
 2030]
['percent']
[6.50000013e-03 4.60000010e-04 2.79999990e-03 2.99999993e-02
 3.40000018e-02 5.40000014e-03 1.70000009e-02 1.59999996e-01
 2.00000003e-01 3.20000015e-02 4.30000015e-02 1.50000006e-01
 2.59999990e-01 6.10000007e-02 8.90000015e-02 4.19999987e-01
 1.20000005e+00 2.19999999e-01 1.10000002e+00 2.79999995e+00
 3.89999986e-01 5.89999974e-01 5.09999990e+00 2.19999999e-02
 7.89999962e-03 1.30000000e-03 1.39999995

In [24]:
df3.head()

,region,category,parameter,mode,powertrain,year,unit,value
40,Australia,Historical,EV charging points,EV,Publicly available fast,2017,charging points,40.0
41,Australia,Historical,EV charging points,EV,Publicly available slow,2017,charging points,440.0
42,Australia,Historical,EV charging points,EV,Publicly available fast,2018,charging points,61.0
43,Australia,Historical,EV charging points,EV,Publicly available slow,2018,charging points,670.0
56,Australia,Historical,EV charging points,EV,Publicly available fast,2019,charging points,250.0


In [25]:
df4.head()

,region,category,parameter,mode,powertrain,year,unit,value
1002,China,Historical,Electricity demand,Buses,EV,2010,GWh,24000.0
1003,China,Historical,Electricity demand,Trucks,EV,2010,GWh,36.0
1004,China,Historical,Electricity demand,Vans,EV,2010,GWh,110.0
1005,China,Historical,Electricity demand,Cars,EV,2010,GWh,330.0
1022,China,Historical,Electricity demand,Buses,EV,2011,GWh,24000.0


In [26]:
df5.head()

,region,category,parameter,mode,powertrain,year,unit,value
1006,China,Historical,Oil displacement Mbd,Buses,EV,2010,Milion barrels per day,0.001100
1007,China,Historical,Oil displacement Mbd,Trucks,EV,2010,Milion barrels per day,0.000002
1008,China,Historical,Oil displacement Mbd,Vans,EV,2010,Milion barrels per day,0.000010
1009,China,Historical,Oil displacement Mbd,Cars,EV,2010,Milion barrels per day,0.000026
1014,China,Historical,Oil displacement Mbd,Buses,EV,2011,Milion barrels per day,0.001100


In [27]:
df6.head()

,region,category,parameter,mode,powertrain,year,unit,value
1010,China,Historical,"Oil displacement, million lge",Buses,EV,2010,"Oil displacement, million lge",68.000
1011,China,Historical,"Oil displacement, million lge",Trucks,EV,2010,"Oil displacement, million lge",0.088
1012,China,Historical,"Oil displacement, million lge",Vans,EV,2010,"Oil displacement, million lge",0.590
1013,China,Historical,"Oil displacement, million lge",Cars,EV,2010,"Oil displacement, million lge",1.500
1018,China,Historical,"Oil displacement, million lge",Buses,EV,2011,"Oil displacement, million lge",67.000


In [28]:
dn=[df1, df2,df3,df4,df5,df6]

In [29]:
df1.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.0
3,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0
4,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0
7,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.0
8,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.0


In [30]:
# from sklearn.preprocessing import OrdinalEncoder
# encoder = OrdinalEncoder()
# encoder.fit(df1[cat_cols])
# df1[cat_cols]= encoder.transform(df1[cat_cols])
# df1.head()
# df1[df1.columns].nunique()


In [31]:
df1

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.0
3,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0
4,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0
7,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.0
8,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.0
...,...,...,...,...,...,...,...,...
9537,World,Projection-APS,EV stock,Trucks,PHEV,2030,Vehicles,910000.0
9538,World,Projection-APS,EV stock,Vans,BEV,2030,Vehicles,12000000.0
9539,World,Projection-APS,EV stock,Vans,PHEV,2030,Vehicles,1300000.0
9540,World,Projection-APS,EV stock,Cars,BEV,2030,Vehicles,180000000.0


In [32]:
# encoder.fit(df2[cat_cols])
# df2[cat_cols]= encoder.transform(df2[cat_cols])
# df2.head()
# df2[df2.columns].nunique()

In [33]:
# encoder.fit(df3[cat_cols])
# df3[cat_cols]= encoder.transform(df3[cat_cols])
# df3.head()
# df3[df3.columns].nunique()

In [34]:
# encoder.fit(df4[cat_cols])
# df4[cat_cols]= encoder.transform(df4[cat_cols])
# df4.head()
# df4[df4.columns].nunique()

In [35]:
# encoder.fit(df5[cat_cols])
# df5[cat_cols]= encoder.transform(df5[cat_cols])
# df5.head()
# df5[df5.columns].nunique()

In [36]:
# encoder.fit(df6[cat_cols])
# df6[cat_cols]= encoder.transform(df6[cat_cols])
# df6.head()
# df6[df6.columns].nunique()

In [37]:
# for cat in df1.columns:
#     IQR=df1[cat].quantile(0.75)-df1[cat].quantile(0.25)
#     UB= df1[cat].quantile(0.75)+ (1.5*IQR)
#     LB= df1[cat].quantile(0.25)-(1.5*IQR)
#     s=0
#     for row in df1[cat]:
#         if row>=UB or row<= LB:
#             s+=1
#     print(f"{cat}: {s}")

In [38]:
# for cat in df1.columns:
#     fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
#     sns.histplot(df1[cat],ax=ax[0],kde=True)
#     ax[0].set_title("Histplot")
#     sns.boxplot(df1[cat],ax=ax[1])
#     ax[1].set_title("Box_Plot")
#     plt.tight_layout()
#     plt.show()

In [39]:
# from scipy.stats import shapiro
# shapiro(df4)

In [40]:
df1.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.0
3,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0
4,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0
7,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.0
8,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.0


In [41]:
cat_cols=[]
num_cols=[]
for cols in df.columns:
    if df[cols].dtype=="object":
        cat_cols.append(cols)
    else:
        num_cols.append(cols)

In [42]:
cat_cols

['region', 'category', 'parameter', 'mode', 'powertrain', 'unit']

In [43]:
from sklearn.model_selection import train_test_split
X=df1.iloc[:,:-1].values
Y=df1[['value']]
X_train , X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=32)

In [44]:
Y

,value
0,49.0
3,49.0
4,170.0
7,220.0
8,80.0
...,...
9537,910000.0
9538,12000000.0
9539,1300000.0
9540,180000000.0


In [45]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4624 entries, 0 to 9541
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      4624 non-null   object 
 1   category    4624 non-null   object 
 2   parameter   4624 non-null   object 
 3   mode        4624 non-null   object 
 4   powertrain  4624 non-null   object 
 5   year        4624 non-null   int64  
 6   unit        4624 non-null   object 
 7   value       4624 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 325.1+ KB


In [46]:
X_train_sp=X_train[:,-2]
X_train_sp

array([2020, 2016, 2018, ..., 2020, 2019, 2021], dtype=object)

In [47]:
X_train_f=X_train[:,:-2]
X_train_s=X_train[:,-1]
X_train_sp=X_train[:,-2]
X_train_sp=np.reshape(X_train_sp,(-1,1))
X_train_s=np.reshape(X_train_s,(-1,1))
X_train_merged = np.concatenate((X_train_f, X_train_s), axis=1)
X_train_merged


X_test_f=X_test[:,:-2]
X_test_s=X_test[:,-1]
X_test_sp=X_test[:,-2]
X_test_sp=np.reshape(X_test_sp,(-1,1))
X_test_s=np.reshape(X_test_s,(-1,1))
X_test_merged = np.concatenate((X_test_f, X_test_s), axis=1)
X_test_merged


array([['Switzerland', 'Historical', 'EV sales', 'Trucks', 'BEV',
        'Vehicles'],
       ['USA', 'Historical', 'EV sales', 'Cars', 'PHEV', 'Vehicles'],
       ['India', 'Historical', 'EV stock', 'Cars', 'PHEV', 'Vehicles'],
       ...,
       ['China', 'Projection-STEPS', 'EV stock', 'Buses', 'BEV',
        'Vehicles'],
       ['Sweden', 'Historical', 'EV sales', 'Cars', 'PHEV', 'Vehicles'],
       ['Denmark', 'Historical', 'EV stock', 'Vans', 'BEV', 'Vehicles']],
      dtype=object)

In [48]:
from sklearn.preprocessing import OrdinalEncoder
encoder =OrdinalEncoder()
X_train_enc= encoder.fit_transform(X_train_merged)
X_train_enc_merge= np.concatenate((X_train_enc, X_train_sp), axis=1)
X_test_enc= encoder.transform(X_test_merged)
X_test_enc_merge= np.concatenate((X_test_enc, X_test_sp), axis=1)


In [49]:
X_train_enc_merge[0]

array([1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 2020], dtype=object)

In [50]:
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler2 = StandardScaler()
X_train_scaled = scaler1.fit_transform(X_train_enc_merge)
X_test_scaled = scaler1.transform(X_test_enc_merge)
# scaler = StandardScaler()
Y_train_scaled= scaler2.fit_transform(Y_train)
Y_test_scaled = scaler2.transform(Y_test)

In [51]:
X_train_scaled[0]

array([-1.5534551 , -0.44316373,  0.9688546 , -0.35914741,  1.29034321,
        0.        ,  0.36844904])

In [52]:
df1.iloc[0:1,:-1].values

array([['Australia', 'Historical', 'EV stock', 'Cars', 'BEV', 2011,
        'Vehicles']], dtype=object)

In [53]:
X_train_scaled[0]

array([-1.5534551 , -0.44316373,  0.9688546 , -0.35914741,  1.29034321,
        0.        ,  0.36844904])

In [54]:
Y_train_scaled

array([[-0.10197648],
       [-0.1052677 ],
       [-0.10545325],
       ...,
       [-0.1052561 ],
       [-0.10406394],
       [-0.09061147]])

In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
model1 = RandomForestRegressor()
model1.fit(X_train_scaled, Y_train_scaled)
Y_pred_scaled= model1.predict(X_test_scaled)
# Y_pred= scaler.inverse_transform(Y_pred_scaled)
eval = mean_squared_error(Y_test_scaled, Y_pred_scaled)
print("mse=",eval)
score = r2_score(Y_test_scaled, Y_pred_scaled)
print("r2_score", score)


/home/ayush/Ayush/Sixth_Sem/ML/myenv/lib/python3.10/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


mse= 0.41618222119349013
r2_score 0.8098946944955479


In [56]:
import joblib 
joblib.dump(scaler1, 'X_scaler1.pkl')
joblib.dump(scaler2, 'Y_scaler1.pkl')
joblib.dump(encoder,'encoder1.pkl')

['encoder1.pkl']

In [57]:
X=df2.iloc[:,:-1].values
Y=df2[['value']]
X_train , X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=32)

X_train_f=X_train[:,:-2]
X_train_s=X_train[:,-1]
X_train_sp=X_train[:,-2]
X_train_sp=np.reshape(X_train_sp,(-1,1))
X_train_s=np.reshape(X_train_s,(-1,1))
X_train_merged = np.concatenate((X_train_f, X_train_s), axis=1)
X_train_merged


X_test_f=X_test[:,:-2]
X_test_s=X_test[:,-1]
X_test_sp=X_test[:,-2]
X_test_sp=np.reshape(X_test_sp,(-1,1))
X_test_s=np.reshape(X_test_s,(-1,1))
X_test_merged = np.concatenate((X_test_f, X_test_s), axis=1)
X_test_merged


from sklearn.preprocessing import OrdinalEncoder
encoder =OrdinalEncoder()
X_train_enc= encoder.fit_transform(X_train_merged)
X_train_enc_merge= np.concatenate((X_train_enc, X_train_sp), axis=1)
X_test_enc= encoder.transform(X_test_merged)
X_test_enc_merge= np.concatenate((X_test_enc, X_test_sp), axis=1)


from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler2 = StandardScaler()
X_train_scaled = scaler1.fit_transform(X_train_enc_merge)
X_test_scaled = scaler1.transform(X_test_enc_merge)
# scaler = StandardScaler()
Y_train_scaled= scaler2.fit_transform(Y_train)
Y_test_scaled = scaler2.transform(Y_test)

model2 = RandomForestRegressor()
model2.fit(X_train_scaled, Y_train_scaled)
Y_pred_scaled= model2.predict(X_test_scaled)
# Y_pred= scaler.inverse_transform(Y_pred_scaled)
eval = mean_squared_error(Y_test_scaled, Y_pred_scaled)
print("mse=",eval)
score = r2_score(Y_test_scaled, Y_pred_scaled)
print("r2_score", score)


/home/ayush/Ayush/Sixth_Sem/ML/myenv/lib/python3.10/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


mse= 0.30260027735562567
r2_score 0.7997879384823109


In [58]:
joblib.dump(scaler1, 'X_scaler2.pkl')
joblib.dump(scaler2, 'Y_scaler2.pkl')
joblib.dump(encoder, 'encoder2.pkl')

['encoder2.pkl']

In [59]:
X=df3.iloc[:,:-1].values
Y=df3[['value']]
X_train , X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=32)
X_train_f=X_train[:,:-2]
X_train_s=X_train[:,-1]
X_train_sp=X_train[:,-2]
X_train_sp=np.reshape(X_train_sp,(-1,1))
X_train_s=np.reshape(X_train_s,(-1,1))
X_train_merged = np.concatenate((X_train_f, X_train_s), axis=1)
X_train_merged


X_test_f=X_test[:,:-2]
X_test_s=X_test[:,-1]
X_test_sp=X_test[:,-2]
X_test_sp=np.reshape(X_test_sp,(-1,1))
X_test_s=np.reshape(X_test_s,(-1,1))
X_test_merged = np.concatenate((X_test_f, X_test_s), axis=1)
X_test_merged


from sklearn.preprocessing import OrdinalEncoder
encoder =OrdinalEncoder()
X_train_enc= encoder.fit_transform(X_train_merged)
X_train_enc_merge= np.concatenate((X_train_enc, X_train_sp), axis=1)
X_test_enc= encoder.transform(X_test_merged)
X_test_enc_merge= np.concatenate((X_test_enc, X_test_sp), axis=1)


from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler2 = StandardScaler()
X_train_scaled = scaler1.fit_transform(X_train_enc_merge)
X_test_scaled = scaler1.transform(X_test_enc_merge)
# scaler = StandardScaler()
Y_train_scaled= scaler2.fit_transform(Y_train)
Y_test_scaled = scaler2.transform(Y_test)

model3 = RandomForestRegressor()
model3.fit(X_train_scaled, Y_train_scaled)
Y_pred_scaled= model3.predict(X_test_scaled)
# Y_pred= scaler.inverse_transform(Y_pred_scaled)
eval = mean_squared_error(Y_test_scaled, Y_pred_scaled)
print("mse=",eval)
score = r2_score(Y_test_scaled, Y_pred_scaled)
print("r2_score", score)


mse= 0.009329209840692711
r2_score 0.6795960339179168


/home/ayush/Ayush/Sixth_Sem/ML/myenv/lib/python3.10/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [60]:
joblib.dump(scaler1, 'X_scaler3.pkl')
joblib.dump(scaler2, 'Y_scaler3.pkl')
joblib.dump(encoder, 'encoder3.pkl')

['encoder3.pkl']

In [61]:
df['unit'].unique()

array(['Vehicles', 'percent', 'charging points', 'GWh',
       'Milion barrels per day', 'Oil displacement, million lge'],
      dtype=object)

In [62]:
X=df4.iloc[:,:-1].values
Y=df4[['value']]
X_train , X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=32)
X_train_f=X_train[:,:-2]
X_train_s=X_train[:,-1]
X_train_sp=X_train[:,-2]
X_train_sp=np.reshape(X_train_sp,(-1,1))
X_train_s=np.reshape(X_train_s,(-1,1))
X_train_merged = np.concatenate((X_train_f, X_train_s), axis=1)
X_train_merged


X_test_f=X_test[:,:-2]
X_test_s=X_test[:,-1]
X_test_sp=X_test[:,-2]
X_test_sp=np.reshape(X_test_sp,(-1,1))
X_test_s=np.reshape(X_test_s,(-1,1))
X_test_merged = np.concatenate((X_test_f, X_test_s), axis=1)
X_test_merged


from sklearn.preprocessing import OrdinalEncoder
encoder =OrdinalEncoder()
X_train_enc= encoder.fit_transform(X_train_merged)
X_train_enc_merge= np.concatenate((X_train_enc, X_train_sp), axis=1)
X_test_enc= encoder.transform(X_test_merged)
X_test_enc_merge= np.concatenate((X_test_enc, X_test_sp), axis=1)


from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler2 = StandardScaler()
X_train_scaled = scaler1.fit_transform(X_train_enc_merge)
X_test_scaled = scaler1.transform(X_test_enc_merge)
# scaler = StandardScaler()
Y_train_scaled= scaler2.fit_transform(Y_train)
Y_test_scaled = scaler2.transform(Y_test)

model4 = RandomForestRegressor()
model4.fit(X_train_scaled, Y_train_scaled)
Y_pred_scaled= model4.predict(X_test_scaled)
# Y_pred= scaler.inverse_transform(Y_pred_scaled)
eval = mean_squared_error(Y_test_scaled, Y_pred_scaled)
print("mse=",eval)
score = r2_score(Y_test_scaled, Y_pred_scaled)
print("r2_score", score)


mse= 4.181292946124825
r2_score 0.479370415773967


/home/ayush/Ayush/Sixth_Sem/ML/myenv/lib/python3.10/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [63]:
joblib.dump(scaler1, 'X_scaler4.pkl')
joblib.dump(scaler2, 'Y_scaler4.pkl')
joblib.dump(encoder, 'encoder4.pkl')

['encoder4.pkl']

In [64]:
X=df5.iloc[:,:-1].values
Y=df5[['value']]
X_train , X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=42)
X_train_f=X_train[:,:-2]
X_train_s=X_train[:,-1]
X_train_sp=X_train[:,-2]
X_train_sp=np.reshape(X_train_sp,(-1,1))
X_train_s=np.reshape(X_train_s,(-1,1))
X_train_merged = np.concatenate((X_train_f, X_train_s), axis=1)
X_train_merged


X_test_f=X_test[:,:-2]
X_test_s=X_test[:,-1]
X_test_sp=X_test[:,-2]
X_test_sp=np.reshape(X_test_sp,(-1,1))
X_test_s=np.reshape(X_test_s,(-1,1))
X_test_merged = np.concatenate((X_test_f, X_test_s), axis=1)
X_test_merged


from sklearn.preprocessing import OrdinalEncoder
encoder =OrdinalEncoder()
X_train_enc= encoder.fit_transform(X_train_merged)
X_train_enc_merge= np.concatenate((X_train_enc, X_train_sp), axis=1)
X_test_enc= encoder.transform(X_test_merged)
X_test_enc_merge= np.concatenate((X_test_enc, X_test_sp), axis=1)


from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler2 = StandardScaler()
X_train_scaled = scaler1.fit_transform(X_train_enc_merge)
X_test_scaled = scaler1.transform(X_test_enc_merge)
# scaler = StandardScaler()
Y_train_scaled= scaler2.fit_transform(Y_train)
Y_test_scaled = scaler2.transform(Y_test)

model5 = RandomForestRegressor()
model5.fit(X_train_scaled, Y_train_scaled)
Y_pred_scaled= model5.predict(X_test_scaled)
# Y_pred= scaler.inverse_transform(Y_pred_scaled)
eval = mean_squared_error(Y_test_scaled, Y_pred_scaled)
print("mse=",eval)
score = r2_score(Y_test_scaled, Y_pred_scaled)
print("r2_score", score)


mse= 0.4737038828533715
r2_score 0.08378384484554047


/home/ayush/Ayush/Sixth_Sem/ML/myenv/lib/python3.10/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [65]:
joblib.dump(scaler1, 'X_scaler5.pkl')
joblib.dump(scaler2, 'Y_scaler5.pkl')
joblib.dump(encoder, 'encoder5.pkl')

['encoder5.pkl']

In [66]:
X=df6.iloc[:,:-1].values
Y=df6[['value']]
X_train , X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=32)

X_train_f=X_train[:,:-2]
X_train_s=X_train[:,-1]
X_train_sp=X_train[:,-2]
X_train_sp=np.reshape(X_train_sp,(-1,1))
X_train_s=np.reshape(X_train_s,(-1,1))
X_train_merged = np.concatenate((X_train_f, X_train_s), axis=1)
X_train_merged


X_test_f=X_test[:,:-2]
X_test_s=X_test[:,-1]
X_test_sp=X_test[:,-2]
X_test_sp=np.reshape(X_test_sp,(-1,1))
X_test_s=np.reshape(X_test_s,(-1,1))
X_test_merged = np.concatenate((X_test_f, X_test_s), axis=1)
X_test_merged


from sklearn.preprocessing import OrdinalEncoder
encoder =OrdinalEncoder()
X_train_enc= encoder.fit_transform(X_train_merged)
X_train_enc_merge= np.concatenate((X_train_enc, X_train_sp), axis=1)
X_test_enc= encoder.transform(X_test_merged)
X_test_enc_merge= np.concatenate((X_test_enc, X_test_sp), axis=1)


from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler2 = StandardScaler()
X_train_scaled = scaler1.fit_transform(X_train_enc_merge)
X_test_scaled = scaler1.transform(X_test_enc_merge)
# scaler = StandardScaler()
Y_train_scaled= scaler2.fit_transform(Y_train)
Y_test_scaled = scaler2.transform(Y_test)

model6 = RandomForestRegressor()
model6.fit(X_train_scaled, Y_train_scaled)
Y_pred_scaled= model6.predict(X_test_scaled)
# Y_pred= scaler.inverse_transform(Y_pred_scaled)
eval = mean_squared_error(Y_test_scaled, Y_pred_scaled)
print("mse=",eval)
score = r2_score(Y_test_scaled, Y_pred_scaled)
print("r2_score", score)


mse= 0.43014353606462524
r2_score 0.3518657690187428


/home/ayush/Ayush/Sixth_Sem/ML/myenv/lib/python3.10/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [67]:
joblib.dump(scaler1, 'X_scaler6.pkl')
joblib.dump(scaler2, 'Y_scaler6.pkl')
joblib.dump(encoder, 'encoder6.pkl')

['encoder6.pkl']

In [68]:
!pwd


/home/ayush/Ayush/Sixth_Sem/ML/ML_project/research


In [69]:
import os
os.chdir("../")

In [70]:
!pwd


/home/ayush/Ayush/Sixth_Sem/ML/ML_project


In [71]:
joblib.dump(model1, 'model1.pkl')
joblib.dump(model2, 'model2.pkl')
joblib.dump(model3, 'model3.pkl')
joblib.dump(model4, 'model4.pkl')
joblib.dump(model5, 'model5.pkl')
joblib.dump(model6, 'model6.pkl')

['model6.pkl']

In [72]:
df1.info

<bound method DataFrame.info of          region        category parameter    mode powertrain  year      unit  \
0     Australia      Historical  EV stock    Cars        BEV  2011  Vehicles   
3     Australia      Historical  EV sales    Cars        BEV  2011  Vehicles   
4     Australia      Historical  EV sales    Cars        BEV  2012  Vehicles   
7     Australia      Historical  EV stock    Cars        BEV  2012  Vehicles   
8     Australia      Historical  EV stock    Cars       PHEV  2012  Vehicles   
...         ...             ...       ...     ...        ...   ...       ...   
9537      World  Projection-APS  EV stock  Trucks       PHEV  2030  Vehicles   
9538      World  Projection-APS  EV stock    Vans        BEV  2030  Vehicles   
9539      World  Projection-APS  EV stock    Vans       PHEV  2030  Vehicles   
9540      World  Projection-APS  EV stock    Cars        BEV  2030  Vehicles   
9541      World  Projection-APS  EV stock    Cars       PHEV  2030  Vehicles   

       